In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import datetime
import os
from typing import Dict, List, Tuple, Optional, Any
from tqdm import tqdm, trange
from detrend_sliding_window import detrend_sliding_window
from flattenDoppler2D import flattenDoppler2D
# from crossvalidate_short import crossvalidate_short
from preprocess import preProcess
import h5py
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Neural decoding analysis script - Python conversion of MATLAB code
# TC: for early direction/value tests with Verasonics, Sessions [164 165 166] - 4 corners 2 values
# simplified script - for Bahareh 20240606

# load a dop_trials_struct before proceeding
def load_matlab_v73(filepath):
    """
    Load MATLAB v7.3 files using h5py
    """
    def h5py_to_dict(h5_file, path='/'):
        """Recursively convert h5py file to dictionary"""
        data = {}
        for key in tqdm(h5_file[path].keys(), desc="Loading MATLAB file structure", leave=False):
            if isinstance(h5_file[path + key], h5py.Dataset):
                # Handle different data types
                dataset = h5_file[path + key]
                if dataset.dtype.kind == 'O':  # Object references
                    # Handle cell arrays or struct arrays
                    try:
                        # Try to dereference object references
                        refs = dataset[:]
                        if refs.ndim == 2 and refs.shape[0] == 1:
                            refs = refs[0]
                        
                        deref_data = []
                        for ref in refs.flat:
                            if ref:
                                deref_data.append(h5_file[ref][:])
                        
                        if len(deref_data) == 1:
                            data[key] = deref_data[0]
                        else:
                            data[key] = deref_data
                    except:
                        # If dereferencing fails, just get the raw data
                        data[key] = dataset[:]
                else:
                    # Regular numeric data
                    arr = dataset[:]
                    # MATLAB stores arrays in column-major order, transpose if needed
                    if arr.ndim > 1:
                        data[key] = arr.T
                    else:
                        data[key] = arr
            elif isinstance(h5_file[path + key], h5py.Group):
                # Recursively handle groups (structs)
                data[key] = h5py_to_dict(h5_file, path + key + '/')
        return data
    
    with h5py.File(filepath, 'r') as f:
        return h5py_to_dict(f)

def debug_data_structure(data_dict, max_depth=3, current_depth=0):
    """Debug function to examine the structure of loaded data"""
    if current_depth > max_depth:
        return
    
    for key, value in data_dict.items():
        indent = "  " * current_depth
        if isinstance(value, dict):
            print(f"{indent}{key}: dict with keys {list(value.keys())}")
            debug_data_structure(value, max_depth, current_depth + 1)
        elif isinstance(value, np.ndarray):
            print(f"{indent}{key}: array shape {value.shape}, dtype {value.dtype}")
        elif isinstance(value, list):
            print(f"{indent}{key}: list of length {len(value)}")
            if len(value) > 0 and isinstance(value[0], np.ndarray):
                print(f"{indent}  First element shape: {value[0].shape}")
        else:
            print(f"{indent}{key}: {type(value)}")

# Neural decoding analysis script - Python conversion of MATLAB code
print("Loading MATLAB file...")
try:
    # First try with h5py for MATLAB v7.3 files
    dop_trials_struct = load_matlab_v73("D:\Downloads\dop_trials_struct_S164_R1_NoRegister_NoDetrend.mat")
    print("Successfully loaded MATLAB v7.3 file with h5py")
except Exception as e:
    print(f"h5py loading failed: {e}")
    try:
        # Fallback to scipy.io.loadmat for older MATLAB files
        from scipy.io import loadmat
        dop_trials_struct = loadmat("D:\Downloads\dop_trials_struct_S164_R1_NoRegister_NoDetrend.mat")
        print("Successfully loaded with scipy.io.loadmat")
    except Exception as e2:
        print(f"Both loading methods failed. h5py error: {e}, scipy error: {e2}")
        raise

# Debug the data structure
print("=== Data Structure Debug ===")
debug_data_structure(dop_trials_struct)

# choose previous N trials for adaptive training set
N = 175
# choose K for K-fold validation
K = 15
# decoding window in seconds
trailing_window = 1

# control for preprocessing steps
time_gain = False
disk_filter = False
z_score = True
detrend = True

# Preprocessing
print("Starting preprocessing...")
if detrend:
    detrend_window_length = 50  # seconds
    print("Applying detrending...")
    iDopP = detrend_sliding_window(dop_trials_struct['dop_trials_struct']['iDopP'], 
                                  detrend_window_length * dop_trials_struct['dop_trials_struct']['acquisition_rate'])
else:
    detrend_window_length = 0
    iDopP = dop_trials_struct['dop_trials_struct']['iDopP']

print(f"iDopP shape before preProcess: {iDopP.shape}")
print(f"iDopP dtype: {iDopP.dtype}")
print(f"Contains NaN: {np.any(np.isnan(iDopP))}")

print("Applying preprocessing...")
iDopP = preProcess(iDopP, timeGain=time_gain, diskFilter=disk_filter, zScore=z_score)

classifier_string = 'PCA+LDA'
validation_string = 'kFold'
protocol_name = 'ValDir'
n_iter = 1

decoding_results = {}
decoding_results['session'] = dop_trials_struct['dop_trials_struct']['session']
decoding_results['run'] = dop_trials_struct['dop_trials_struct']['run']
decoding_results['protocol_name'] = protocol_name

# indexing labels
only_use_successful_trials = 1
axes_to_decode = []  # [] for both, 1 for horz, 2 for vert

fixation_pos = dop_trials_struct['dop_trials_struct']['fixation_pos']
target_cue_pos = np.squeeze(dop_trials_struct['dop_trials_struct']['target_pos_eachCue']).T
tprime_cue_pos = np.squeeze(dop_trials_struct['dop_trials_struct']['targetPrime_pos_eachCue']).T

success_trial_idx = np.isin(dop_trials_struct['dop_trials_struct']['reached_minimum_state_idx'].flatten(),
                           dop_trials_struct['dop_trials_struct']['success_idx'].flatten())

decoding_results['only_use_successful_trials'] = only_use_successful_trials
decoding_results['axes_to_decode'] = axes_to_decode

print(f"Starting {n_iter} iteration(s) of decoding analysis...")

for nn in trange(n_iter, desc="Processing iterations"):
    # for saccade direction ipsi/contra or up/down
    label_horz = np.sign(target_cue_pos[:, 0])
    label_vert = np.sign(target_cue_pos[:, 1])

    # set to min of 1
    label_horz = label_horz - np.min(label_horz) + 1
    label_vert = label_vert - np.min(label_vert) + 1

    if only_use_successful_trials:
        label_horz[~success_trial_idx] = np.nan
        label_vert[~success_trial_idx] = np.nan

    # equalize counts of each label combination
    keepidx = []
    mincount_combo = 10000
    
    if len(axes_to_decode) == 0:
        unique_horz = np.unique(label_horz[~np.isnan(label_horz)])
        unique_vert = np.unique(label_vert[~np.isnan(label_vert)])
    elif axes_to_decode == 1:
        unique_horz = np.unique(label_horz[~np.isnan(label_horz)])
        unique_vert = [1]
    elif axes_to_decode == 2:
        unique_horz = [1]
        unique_vert = np.unique(label_vert[~np.isnan(label_vert)])

    # find minimum count of combos
    print("Finding minimum count of label combinations...")
    for u1 in tqdm(range(len(unique_horz)), desc="Processing horizontal labels", leave=False):
        for u2 in range(len(unique_vert)):
            if len(axes_to_decode) == 0:
                idx = (label_horz == unique_horz[u1]) & (label_vert == unique_vert[u2])
            elif axes_to_decode == 1:
                idx = label_horz == unique_horz[u1]
            elif axes_to_decode == 2:
                idx = label_vert == unique_vert[u2]
            mincount_combo = min(mincount_combo, np.sum(idx))

    # create set with equal representation all combos
    print("Creating balanced dataset...")
    for u1 in tqdm(range(len(unique_horz)), desc="Balancing labels", leave=False):
        for u2 in range(len(unique_vert)):
            if len(axes_to_decode) == 0:
                idx = (label_horz == unique_horz[u1]) & (label_vert == unique_vert[u2])
            elif axes_to_decode == 1:
                idx = label_horz == unique_horz[u1]
            elif axes_to_decode == 2:
                idx = label_vert == unique_vert[u2]

            idx = np.where(idx)[0]
            keepidx.extend(np.random.choice(idx, mincount_combo, replace=False))

    keepidx = np.sort(keepidx)

    train_labels_horz = label_horz[keepidx]
    train_labels_vert = label_vert[keepidx]

    print(f"train_labels_horz shape: {train_labels_horz.shape}")

    # FIX: Handle the lookup_tables_3D_to_4D structure more robustly
    lookup_table = dop_trials_struct['dop_trials_struct']['lookup_tables_3D_to_4D']
    
    print(f"lookup_tables_3D_to_4D type: {type(lookup_table)}")
    if isinstance(lookup_table, list):
        print(f"lookup_tables_3D_to_4D is a list of length: {len(lookup_table)}")
        if len(lookup_table) > 0:
            lookup_array = lookup_table[0]
            print(f"First element shape: {lookup_array.shape}")
        else:
            raise ValueError("lookup_tables_3D_to_4D list is empty")
    else:
        lookup_array = lookup_table
        print(f"lookup_tables_3D_to_4D shape: {lookup_array.shape}")
    
    # Handle different possible shapes
    if lookup_array.ndim == 1:
        # If 1D, assume it's a single trial's time indices
        trial_n_timestamps = len(lookup_array)
        nTrials = 1
        print(f"Detected 1D lookup table: {trial_n_timestamps} timestamps, 1 trial")
    elif lookup_array.ndim == 2:
        trial_n_timestamps = lookup_array.shape[0]
        nTrials = lookup_array.shape[1]
        print(f"Detected 2D lookup table: {trial_n_timestamps} timestamps, {nTrials} trials")
    else:
        raise ValueError(f"Unexpected lookup table dimensions: {lookup_array.shape}")

    yPix, xPix, nWindows = iDopP.shape

    # create doppler struct that is (yPix, xPix, nWindows, nTrials) aligned to event
    print("Creating training data structure...")
    train_data = np.full((yPix, xPix, trial_n_timestamps, nTrials), np.nan)
    print(train_data.shape)
    
    for tt in tqdm(range(nTrials), desc="Processing trials", leave=False):
        if lookup_array.ndim == 1:
            # Single trial case
            timeidx = lookup_array
        else:
            # Multiple trials case
            timeidx = lookup_array[:, tt]
            
        if not np.all(np.isnan(timeidx)):
            # Ensure indices are valid
            valid_indices = timeidx[~np.isnan(timeidx)].astype(int)
            valid_indices = valid_indices[valid_indices < nWindows]  # Bounds check
            
            if len(valid_indices) > 0:
                if lookup_array.ndim == 1:
                    train_data[:, :, :len(valid_indices), tt] = iDopP[:, :, valid_indices]
                else:
                    train_data[:, :, :len(valid_indices), tt] = iDopP[:, :, valid_indices]

    train_data = train_data.reshape(yPix * xPix, trial_n_timestamps, nTrials)
    
    # Handle case where nTrials might be 1 but we need multiple trials from keepidx
    if nTrials == 1 and len(keepidx) > 1:
        print("Warning: Only 1 trial in lookup table but multiple trials needed for training")
        print("This might indicate a problem with the data structure")
        # Use the available trial for all keepidx indices (not ideal but prevents crash)
        train_data = np.repeat(train_data, len(keepidx), axis=2)
    else:
        train_data = train_data[:, :, keepidx] if len(keepidx) <= nTrials else train_data

    # decode across trial time
    nWindows = train_data.shape[1]

    # Determine dimensionality of cPCA subspace (if used)
    m = len(np.unique(train_labels_horz)) - 1

    # allocate memory
    result_horz = [None] * nWindows
    confusion_horz = [None] * nWindows
    cp_horz = [None] * nWindows
    class_predicted_horz = [None] * nWindows
    p_horz = np.full(nWindows, np.nan)
    percentCorrect_horz = np.full(nWindows, np.nan)
    t_horz = np.full(nWindows, np.nan)

    result_vert = [None] * nWindows
    confusion_vert = [None] * nWindows
    cp_vert = [None] * nWindows
    class_predicted_vert = [None] * nWindows
    p_vert = np.full(nWindows, np.nan)
    percentCorrect_vert = np.full(nWindows, np.nan)
    t_vert = np.full(nWindows, np.nan)

    downsample_interval = 1  # in seconds
    decoding_time_stepsize = 1  # in seconds

    # set decoding start and stop time
    decoding_start_time = decoding_time_stepsize
    decoding_end_time = nWindows

    acquisition_rate = dop_trials_struct['dop_trials_struct']['acquisition_rate']

    if (downsample_interval * acquisition_rate) % 1 != 0:
        raise ValueError('error: the downsample_interval must correspond to an integer number of frames')
    if (decoding_time_stepsize * acquisition_rate) % 1 != 0:
        raise ValueError('error: the decoding_time_stepsize must correspond to an integer number of frames')
    if (trailing_window * acquisition_rate) % 1 != 0:
        raise ValueError('error: the trailingwindow must correspond to an integer number of frames')
    
    trailing_window = int(trailing_window * acquisition_rate)

    trial_time = dop_trials_struct['dop_trials_struct']['trial_timevec'][0]

    print('Starting time-resolved decoding...')
    variance_to_keep = 95
    acq_rate_scalar = float(acquisition_rate)

    # Calculate the total number of time points for progress bar
    time_points = list(range(int(decoding_start_time * acquisition_rate), 
                            int(decoding_end_time), 
                            int(decoding_time_stepsize * acquisition_rate)))
    
    for ii in tqdm(time_points, desc=f"Decoding timepoints (iter {nn+1}/{n_iter})"):
        
        test_time_i = list(range(max(0, ii - trailing_window), ii + 1))

        # prepare training data for this time window
        train_data_window = []

        if len(test_time_i) < (downsample_interval * acquisition_rate):
            # Not enough data — use mean over available
            downsample_data = np.mean(train_data[:, test_time_i, :], axis=1)
            train_data_window.append(flattenDoppler2D(downsample_data, 1))
        else:
            # Main downsampling loop
            for jj in range(test_time_i[0] + int(downsample_interval * acquisition_rate) - 1,
                        test_time_i[-1] + 1,
                        int(downsample_interval * acquisition_rate)):
                downsample_idx = list(range(jj - int(downsample_interval * acquisition_rate) + 1, jj + 1))

                downsample_data = np.mean(train_data[:, test_time_i, :], axis=1)  # shape: (yPix * xPix, nTrials)

                # Reshape properly using original yPix and xPix:
                downsample_data_reshaped = downsample_data.reshape((yPix, xPix, 1, train_data.shape[2]))

                # Now call flattenDoppler2D
                train_data_window.append(flattenDoppler2D(downsample_data_reshaped, 0))

        # Concatenate time windows
        if train_data_window:
            train_data_final = np.concatenate(train_data_window, axis=1) if len(train_data_window) > 1 else train_data_window[0]
        else:
            continue

        # === Horizontal decoding ===
        if len(axes_to_decode) == 0 or axes_to_decode == 1:
            cp_horz[ii], p_horz[ii], class_predicted_horz[ii] = crossvalidate_short(
                train_data_final, train_labels_horz,
                classificationMethod=classifier_string, N=N,
                validationMethod=validation_string, K=K,
                m=m, variance_to_keep=variance_to_keep
            )

            percentCorrect_horz[ii] = cp_horz[ii]['CorrectRate'] * 100
            confusion_horz[ii] = cp_horz[ii]['CountingMatrix']

        # === Vertical decoding ===
        if len(axes_to_decode) == 0 or axes_to_decode == 2:
            cp_vert[ii], p_vert[ii], class_predicted_vert[ii] = crossvalidate_short(
                train_data_final, train_labels_vert,
                classificationMethod=classifier_string, N=N,
                validationMethod=validation_string, K=K,
                m=m, variance_to_keep=variance_to_keep
            )
            percentCorrect_vert[ii] = cp_vert[ii]['CorrectRate'] * 100
            confusion_vert[ii] = cp_vert[ii]['CountingMatrix']

        # Save current result
        result_horz[ii] = {
            'percentCorrect': percentCorrect_horz[ii],
            'confusion': confusion_horz[ii],
            'p': p_horz[ii],
            't': ii / acquisition_rate
        }

        result_vert[ii] = {
            'percentCorrect': percentCorrect_vert[ii],
            'confusion': confusion_vert[ii],
            'p': p_vert[ii],
            't': ii / acquisition_rate
        }

    # ==== Clean up results ====
    print("Cleaning up results...")

    elim_idx_downsampling = [i for i, x in enumerate(result_horz) if x is None]

    result_horz = [x for i, x in enumerate(result_horz) if i not in elim_idx_downsampling]
    result_vert = [x for i, x in enumerate(result_vert) if i not in elim_idx_downsampling]

    percentCorrect_horz = np.array([x['percentCorrect'] for x in result_horz])
    confusion_horz = [x['confusion'] for x in result_horz]
    p_horz = np.array([x['p'] for x in result_horz])
    t_horz = np.array([x['t'] for x in result_horz])

    percentCorrect_vert = np.array([x['percentCorrect'] for x in result_vert])
    confusion_vert = [x['confusion'] for x in result_vert]
    p_vert = np.array([x['p'] for x in result_vert])
    t_vert = np.array([x['t'] for x in result_vert])

    # Store results
    decoding_results[f'result_horz_{nn}'] = result_horz
    decoding_results[f'result_vert_{nn}'] = result_vert
    decoding_results[f'percentCorrect_horz_{nn}'] = percentCorrect_horz
    decoding_results[f'confusion_horz_{nn}'] = confusion_horz
    decoding_results[f'p_horz_{nn}'] = p_horz
    decoding_results[f't_horz_{nn}'] = t_horz

    decoding_results[f'percentCorrect_vert_{nn}'] = percentCorrect_vert
    decoding_results[f'confusion_vert_{nn}'] = confusion_vert
    decoding_results[f'p_vert_{nn}'] = p_vert
    decoding_results[f't_vert_{nn}'] = t_vert

    print(f'Completed iteration {nn+1}/{n_iter}')

print('Finished decoding! Starting visualization...')

# Plotting
percentCorrect_horz = decoding_results['percentCorrect_horz_0']
percentCorrect_vert = decoding_results['percentCorrect_vert_0']
p_horz = decoding_results['p_horz_0']
p_vert = decoding_results['p_vert_0']
t_horz = decoding_results['t_horz_0']
t_vert = decoding_results['t_vert_0']

# Get labels for chance level calculation
unique_horz_labels = len(np.unique(train_labels_horz[~np.isnan(train_labels_horz)]))
unique_vert_labels = len(np.unique(train_labels_vert[~np.isnan(train_labels_vert)]))

p_threshold = 0.05
xdat = t_horz

plt.figure(figsize=(10, 6))

ydat = percentCorrect_horz
pdat = p_horz
chance_level = 100 / unique_horz_labels
best_idx_horz = np.argmax(ydat)
plt.plot(xdat, ydat, linewidth=2, color='k', label='horizontal direction')

if np.any(pdat < p_threshold / len(xdat)):
    significant_points = xdat[pdat < p_threshold / len(xdat)]
    plt.scatter(significant_points, np.full_like(significant_points, 99), 
               marker='*', color='k', s=50)

plt.axhline(y=chance_level, linestyle='--', color='k')
plt.text(15, 20, 'horizontal direction', color='k', fontsize=11)

ydat = percentCorrect_vert
pdat = p_vert
chance_level_vert = 100 / unique_vert_labels

if chance_level_vert < 100:
    best_idx_vert = np.argmax(ydat)
    plt.plot(xdat, ydat, linewidth=2, color='b', label='vertical direction')
    
    if np.any(pdat < p_threshold / len(xdat)):
        significant_points = xdat[pdat < p_threshold / len(xdat)]
        plt.scatter(significant_points, np.full_like(significant_points, 98), 
                   marker='*', color='b', s=50)
    
    plt.axhline(y=chance_level_vert, linestyle='--', color='b')
    plt.text(15, 16, 'vertical direction', color='b', fontsize=11)

plt.xlabel('time (s)', fontsize=15)
plt.ylabel('% correct', fontsize=15)
plt.title(f'session {decoding_results["session"]}', fontsize=15)

plt.xlim([xdat[0], xdat[-1]])
plt.ylim([0, 100])

plt.axvline(x=0, color='b')
plt.text(0.3, 80, 'Cue', color='b', fontsize=11)

plt.tight_layout()
plt.show()

print("Analysis complete!")

# Uncomment to save results
# print("Saving results...")
# path = f'C:/Users/sanvi/Documents/MATLAB/behavioral_decoding/results/{decoding_results["session"]}'
# os.makedirs(path, exist_ok=True)
# savetime = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
# processing = 'NoRegister_NoDetrend'
# savename = f'decodingResults_{protocol_name}_S{decoding_results["session"]}_{savetime}_{processing}'
# full_path = os.path.join(path, savename)
# savemat(full_path + '.mat', {'decodingResults': decoding_results})
# plt.savefig(os.path.join(path, f'{processing}.png'))
# print(f"Results saved to: {full_path}")

Loading MATLAB file...


Loading MATLAB file structure:   0%|          | 0/3 [00:00<?, ?it/s]





































































































































































































































































































































































































































































































































































































































































































































































































































































































































































Successfully loaded MATLAB v7.3 file with h5py
=== Data Structure Debug ===
#refs#: dict with keys ['0', '00', '00b', '00c', '00d', '01', '01b', '01c', '01d', '02', '02b', '02c', '02d', '03', '03b', '03c', '03d', '04', '04b', '04c', '04d', '05', '05b', '05c', '05d', '06', '06b', '06c', '06d', '07', '07b', '07c', '07d', '08', '08b', '08c', '08d', '09', '09b', '09c', '09d', '0A', '0Ab', '0Ac', '0Ad', '0Ae', '0B', '0Bb', '0Bc', '0Bd', '0Be', '0C', '0Cb', '0Cc', '0Cd', '0Ce', '0D', '0Db', '0Dc', '0Dd', '0De', '0E', '0Eb', '0Ec', '0Ed', '0Ee', '0F', '0Fb', '0Fc', '0Fd', '0Fe', '0G', '0Gb', '0Gc', '0Gd', '0Ge', '0H', '0Hb', '0Hc', '0Hd', '0He', '0I', '0Ib', '0Ic', '0Id', '0Ie', '0J', '0Jb', '0Jc', '0Jd', '0Je', '0K', '0Kb', '0Kc', '0Kd', '0Ke', '0L', '0Lb', '0Lc', '0Ld', '0Le', '0M', '0Mb', '0Mc', '0Md', '0Me', '0N', '0Nb', '0Nc', '0Nd', '0Ne', '0O', '0Ob', '0Oc', '0Od', '0P', '0Pb', '0Pc', '0Pd', '0Q', '0Qb', '0Qc', '0Qd', '0R', '0Rb', '0Rc', '0Rd', '0S', '0Sb', '0Sc', '0Sd', '0T', '0Tb', '

Processing iterations:   0%|          | 0/1 [00:00<?, ?it/s]

Finding minimum count of label combinations...


Creating balanced dataset...


train_labels_horz shape: (308,)
lookup_tables_3D_to_4D type: <class 'numpy.ndarray'>
lookup_tables_3D_to_4D shape: (427, 25)
Detected 2D lookup table: 427 timestamps, 25 trials
Creating training data structure...
(133, 128, 427, 25)


Starting time-resolved decoding...


Processing iterations:   0%|          | 0/1 [11:11<?, ?it/s]


KeyboardInterrupt: 

In [1]:
import numpy as np
from scipy.stats import zscore
from sklearn.model_selection import KFold
from scipy.stats import binom
from binomialTest import binomial_test
from classifyDoppler_short import classifyDoppler_short


def crossvalidate_short(data, labels, verbose=False, validationMethod='kFold', K=10, N=30,
                       classificationMethod='PCA+LDA', testData=None, m=1, variance_to_keep=95,
                       trial_ind=None):
    """
    crossValidate will load data from a set of session/run combinations
    and then cross validate those results using your method of choice
    
    Parameters:
    data: array of shape (nImagesL&R x nPixels*yPixels)
    labels: array of shape (nImages,) with values like [0, 1, 0, 0, 1, 1, ...]
    verbose: boolean - print results
    validationMethod: string - options are 'kFold'
    K: integer - for K-fold validation (default: 10)
    N: integer - parameter for validation (default: 30)
    classificationMethod: string - options are 'PCA+LDA'
    testData: array of same shape as data - optional test data from different time window
    m: positive integer - subspace dimensions for cPCA (max: # classes - 1, default: 1)
    variance_to_keep: float - variance to keep in PCA (default: 95)
    trial_ind: array - trial indices (default: all trials)
    
    Returns:
    cp: dictionary containing classification performance metrics
    p: p-value from binomial test
    class_tracker: array of predicted classes for each trial
    """
    
    # Handle default trial indices
    if trial_ind is None:
        trial_ind = np.arange(len(labels))
    
    # Normalize to z-score
    data = zscore(data, axis=0)  # standardize features
    if testData is not None:
        testData = zscore(testData, axis=0)
    
    # Get useful variables
    N_trials = data.shape[0]  # number of trials
    
    # Initialize class performance tracking
    cp = {
        'CorrectRate': 0.0,
        'CountingMatrix': np.zeros((len(np.unique(labels)), len(np.unique(labels)))),
        'ErrorRate': 0.0,
        'LastClassName': None,
        'ClassNames': np.unique(labels),
        'TestTargets': [],
        'TestResults': []
    }
    
    # k-fold cross validation
    if validationMethod == 'kFold':
        # Create k-fold indices
        kfold = KFold(n_splits=K, shuffle=True, random_state=42)
        class_tracker = np.full(N_trials, np.nan)
        
        fold_idx = 0
        # For each k-fold
        for train_idx, test_idx in kfold.split(data):
            fold_idx += 1
            
            # Get training and test data
            train_data = data[train_idx]
            train_labels = labels[train_idx]
            
            if testData is not None:
                test_data = testData[test_idx]
            else:
                test_data = data[test_idx]
            
            test_labels = labels[test_idx]
            
            # Classify using your classification function
            # Note: You need to implement classifyDoppler_short
            predicted_classes, model = classifyDoppler_short(
                train_data, train_labels, test_data,
                method=classificationMethod,
                m=m,
                variance_to_keep=variance_to_keep
            )
            
            # Update class performance
            classperf_update(cp, predicted_classes, test_labels)
            class_tracker[test_idx] = predicted_classes
            
            # Optional progress tracking
            # print(f'Finished fold {fold_idx}/{K}')
    
    # Calculate classification accuracy measures
    nCorrect = np.sum(np.diag(cp['CountingMatrix']))
    nCounted = np.sum(cp['CountingMatrix'])
    
    if nCounted > 0:
        cp['CorrectRate'] = nCorrect / nCounted
        cp['ErrorRate'] = 1 - cp['CorrectRate']
    
    percentCorrect = cp['CorrectRate'] * 100
    chance = 1 / len(np.unique(labels))
    
    # Binomial test for significance
    p = binomial_test(nCorrect, nCounted, chance, 'one')
    
    # Display measures if verbose is on
    if verbose:
        print(f'\nClassification Accuracy:')
        print(f'{int(nCorrect)} / {int(nCounted)} trials correctly classified ({percentCorrect:.2f}% correct)\t', end='')
        
        if p < 0.001:
            print('(binomial test: p < 0.001)')
        else:
            print(f'(binomial test: p = {p:.3f})')
    
    return cp, p, class_tracker


def classperf_update(cp, predicted_classes, true_labels):
    """
    Update classification performance metrics
    Equivalent to MATLAB's classperf function
    """
    # Convert to numpy arrays
    predicted_classes = np.array(predicted_classes).flatten()
    true_labels = np.array(true_labels)
    
    # Store test results
    cp['TestTargets'].extend(true_labels.tolist())
    cp['TestResults'].extend(predicted_classes.tolist())
    
    # Update confusion matrix
    unique_labels = cp['ClassNames']
    for i, true_label in enumerate(unique_labels):
        for j, pred_label in enumerate(unique_labels):
            mask_true = (true_labels == true_label)
            mask_pred = (predicted_classes == pred_label)
            cp['CountingMatrix'][i, j] += np.sum(mask_true & mask_pred)

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
from datetime import datetime
import os
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from detrend_sliding_window import detrend_sliding_window
from crossvalidate_short import crossvalidate_short
from preprocess import preProcess
from flattenDoppler2D import flattenDoppler2D
import warnings
warnings.filterwarnings('ignore')
import h5py


# TC: for early direction/value tests with Verasonics, Sessions [164 165 166] - 4 corners 2 values
# simplified script - for Bahareh 20240606

# load a dop_trials_struct before proceeding
def load_matlab_v73(filepath):
    """
    Load MATLAB v7.3 files using h5py
    """
    def h5py_to_dict(h5_file, path='/'):
        """Recursively convert h5py file to dictionary"""
        data = {}
        for key in h5_file[path].keys():
            if isinstance(h5_file[path + key], h5py.Dataset):
                # Handle different data types
                dataset = h5_file[path + key]
                if dataset.dtype.kind == 'O':  # Object references
                    # Handle cell arrays or struct arrays
                    try:
                        # Try to dereference object references
                        refs = dataset[:]
                        if refs.ndim == 2 and refs.shape[0] == 1:
                            refs = refs[0]
                        
                        deref_data = []
                        for ref in refs.flat:
                            if ref:
                                deref_data.append(h5_file[ref][:])
                        
                        if len(deref_data) == 1:
                            data[key] = deref_data[0]
                        else:
                            data[key] = deref_data
                    except:
                        # If dereferencing fails, just get the raw data
                        data[key] = dataset[:]
                else:
                    # Regular numeric data
                    arr = dataset[:]
                    # MATLAB stores arrays in column-major order, transpose if needed
                    if arr.ndim > 1:
                        data[key] = arr.T
                    else:
                        data[key] = arr
            elif isinstance(h5_file[path + key], h5py.Group):
                # Recursively handle groups (structs)
                data[key] = h5py_to_dict(h5_file, path + key + '/')
        return data
    
    with h5py.File(filepath, 'r') as f:
        return h5py_to_dict(f)

def debug_data_structure(data_dict, max_depth=3, current_depth=0):
    """Debug function to examine the structure of loaded data"""
    if current_depth > max_depth:
        return
    
    for key, value in data_dict.items():
        indent = "  " * current_depth
        if isinstance(value, dict):
            print(f"{indent}{key}: dict with keys {list(value.keys())}")
            debug_data_structure(value, max_depth, current_depth + 1)
        elif isinstance(value, np.ndarray):
            print(f"{indent}{key}: array shape {value.shape}, dtype {value.dtype}")
        elif isinstance(value, list):
            print(f"{indent}{key}: list of length {len(value)}")
            if len(value) > 0 and isinstance(value[0], np.ndarray):
                print(f"{indent}  First element shape: {value[0].shape}")
        else:
            print(f"{indent}{key}: {type(value)}")

# Neural decoding analysis script - Python conversion of MATLAB code
try:
    # First try with h5py for MATLAB v7.3 files
    dop_trials_struct = load_matlab_v73(r"D:\Downloads\dop_trials_struct_S164_R1_NoRegister_NoDetrend.mat")
    print("Successfully loaded MATLAB v7.3 file with h5py")
except Exception as e:
    print(f"h5py loading failed: {e}")
    try:
        # Fallback to scipy.io.loadmat for older MATLAB files
        from scipy.io import loadmat
        dop_trials_struct = loadmat(r"D:\Downloads\dop_trials_struct_S164_R1_NoRegister_NoDetrend.mat")
        print("Successfully loaded with scipy.io.loadmat")
    except Exception as e2:
        print(f"Both loading methods failed. h5py error: {e}, scipy error: {e2}")
        raise

# Debug the data structure
print("=== Data Structure Debug ===")
debug_data_structure(dop_trials_struct)

processing = 'NoRegister_NoDetrend'

# choose previous N trials for adaptive training set
N = 175
# choose K for K-fold validation
K = 15
# decoding window in seconds
trailingwindow = 4

# control for preprocessing steps

# Fits an exponential model to the mean signal across depth and uses the
# model to normalize the data. Makes sure that signals from deeper regions
# are properly scaled.
timeGain = False

# Averaging filter that smooths each 2D slice (maps to each window-trial)
# from input data
diskFilter = False

# zscore is applied to every voxel.
zScore = True

# Removes low frequency trends (tissue movement/environment influences)
# from the Doppler data using a sliding window approach of 50 seconds.
detrend = False

# Question: Are windows overlapping?
# Clarify: We are averaging?
if detrend:
    detrend_window_length = 50  # seconds
    iDopP = detrend_sliding_window(dop_trials_struct['dop_trials_struct']['iDopP'], 
                                  detrend_window_length * dop_trials_struct['dop_trials_struct']['acquisition_rate'])
else:
    detrend_window_length = 0
    iDopP = dop_trials_struct['dop_trials_struct']['iDopP']
    
# if detrend:
#     detrend_window_length = 50  # seconds
#     # Access data based on whether it's h5py loaded or scipy loaded
#     if isinstance(dop_trials_struct, dict) and 'iDopP' in dop_trials_struct:
#         iDopP_data = dop_trials_struct['dop_trials_struct']['iDopP']
#         acq_rate = dop_trials_struct['dop_trials_struct']['acquisition_rate']
#     else:
#         # Fallback for scipy.io loaded data
#         iDopP_data = dop_trials_struct['dop_trials_struct']['iDopP'][0,0]
#         acq_rate = dop_trials_struct['dop_trials_struct']['acquisition_rate'][0,0][0,0]
    
#     iDopP = detrend_sliding_window(iDopP_data, detrend_window_length * acq_rate)
# else:
#     detrend_window_length = 0
#     # Access data based on loading method
#     if isinstance(dop_trials_struct, dict) and 'iDopP' in dop_trials_struct:
#         iDopP = dop_trials_struct['dop_trials_struct']['iDopP']
#     else:
#         iDopP = dop_trials_struct['dop_trials_struct']['iDopP'][0,0]

# Performs these preprocessing steps
print(f"iDopP shape before preProcess: {iDopP.shape}")
print(f"iDopP dtype: {iDopP.dtype}")
print(f"Contains NaN: {np.any(np.isnan(iDopP))}")
iDopP = preProcess(iDopP, timeGain=timeGain, diskFilter=diskFilter, zScore=zScore)

# No Register - this means no motion correction but yes detrend
# No Register + No Detrend - no motion correction and no detrend
# Normcorre - motion correction...no detrend (??)

# Now we do PCA + LDA.

# PCA does dimensionality reduction by transforming the
# original variables to new set of uncorrelated variables that capture most
# variance. PCA also does noise reduction as it focuses on the principal
# components that capture the most variance, and inform the patterns in
# brain signals

# LDA separates the classes in the best way possible as it finds the
# discriminants that maximizes the distance between classes

classifierString = 'PCA+LDA'
validationString = 'kFold'

protocol_name = 'ValDir'
n_iter = 1

decodingResults = {}
decodingResults['session'] = dop_trials_struct['dop_trials_struct'].get('session', 0)
decodingResults['run'] = dop_trials_struct['dop_trials_struct'].get('run', 0)
decodingResults['protocol_name'] = protocol_name

## indexing labels

# manually set
only_use_successful_trials = 1
axes_to_decode = []  # [] for both, 1 for horz, 2 for vert


fixation_pos = dop_trials_struct['dop_trials_struct']['fixation_pos']
target_cue_pos = np.squeeze(dop_trials_struct['dop_trials_struct']['target_pos_eachCue']).T
tprime_cue_pos = np.squeeze(dop_trials_struct['dop_trials_struct']['targetPrime_pos_eachCue']).T
    
# reached_min_state = dop_trials_struct['dop_trials_struct'].get('reached_minimum_state_idx', np.array([]))
# success_idx = dop_trials_struct['dop_trials_struct'].get('success_idx', np.array([]))
    
success_trial_idx = np.isin(dop_trials_struct['dop_trials_struct']['reached_minimum_state_idx'].flatten(),
                           dop_trials_struct['dop_trials_struct']['success_idx'].flatten())

decodingResults['only_use_successful_trials'] = only_use_successful_trials
decodingResults['axes_to_decode'] = axes_to_decode

for nn in range(n_iter):
    # for saccade direction ipsi/contra or up/down
    label_horz = np.sign(target_cue_pos[:, 0])
    label_vert = np.sign(target_cue_pos[:, 1])
    
    # set to min of 1
    label_horz = label_horz - np.min(label_horz) + 1
    label_vert = label_vert - np.min(label_vert) + 1
    
    if only_use_successful_trials:
        # FIX: Handle the indexing properly based on the actual shape of success_trial_idx
        if success_trial_idx.ndim == 1:
            # If success_trial_idx is 1D, use it directly
            label_horz[~success_trial_idx] = np.nan
            label_vert[~success_trial_idx] = np.nan
        elif success_trial_idx.ndim == 2:
            # If success_trial_idx is 2D, use the first column as originally intended
            label_horz[~success_trial_idx[:, 0]] = np.nan
            label_vert[~success_trial_idx[:, 0]] = np.nan
        else:
            print(f"Warning: Unexpected success_trial_idx shape: {success_trial_idx.shape}")
            # Use flattened version as fallback
            success_trial_idx_flat = success_trial_idx.flatten()
            if len(success_trial_idx_flat) == len(label_horz):
                label_horz[~success_trial_idx_flat] = np.nan
                label_vert[~success_trial_idx_flat] = np.nan
            else:
                print("Cannot match success_trial_idx to labels, skipping successful trials filter")


    # if only_use_successful_trials:
    #     label_horz[~success_trial_idx[:, 0]] = np.nan
    #     label_vert[~success_trial_idx[:, 0]] = np.nan
    
    # equalize counts of each label combination
    keepidx = []
    mincount_combo = 10000
    
    if len(axes_to_decode) == 0:
        unique_horz = np.unique(label_horz[~np.isnan(label_horz)])
        unique_vert = np.unique(label_vert[~np.isnan(label_vert)])
    elif axes_to_decode == 1:
        unique_horz = np.unique(label_horz[~np.isnan(label_horz)])
        unique_vert = [1]
    elif axes_to_decode == 2:
        unique_horz = [1]
        unique_vert = np.unique(label_vert[~np.isnan(label_vert)])
    
    # find minimum count of combos
    for u1 in range(len(unique_horz)):
        for u2 in range(len(unique_vert)):
            if len(axes_to_decode) == 0:
                idx = (label_horz == unique_horz[u1]) & (label_vert == unique_vert[u2])
            elif axes_to_decode == 1:
                idx = label_horz == unique_horz[u1]
            elif axes_to_decode == 2:
                idx = label_vert == unique_vert[u2]
            mincount_combo = min(mincount_combo, np.sum(idx))
    
    # create set with equal representation all combos
    for u1 in range(len(unique_horz)):
        for u2 in range(len(unique_vert)):
            if len(axes_to_decode) == 0:
                idx = (label_horz == unique_horz[u1]) & (label_vert == unique_vert[u2])
            elif axes_to_decode == 1:
                idx = label_horz == unique_horz[u1]
            elif axes_to_decode == 2:
                idx = label_vert == unique_vert[u2]
            
            idx_indices = np.where(idx)[0]
            selected_indices = np.random.choice(idx_indices, mincount_combo, replace=False)
            keepidx.extend(selected_indices)
    
    keepidx = np.sort(keepidx)
    
    train_labels_horz = label_horz[keepidx]
    train_labels_vert = label_vert[keepidx]
    
    print(f"train_labels_horz shape: {train_labels_horz.shape}")
    
    lookup_table = dop_trials_struct['dop_trials_struct']['lookup_tables_3D_to_4D']

    print(f"lookup_tables_3D_to_4D type: {type(lookup_table)}")
    if isinstance(lookup_table, list):
        print(f"lookup_tables_3D_to_4D is a list of length: {len(lookup_table)}")
        if len(lookup_table) > 0:
            lookup_array = lookup_table[0]
            print(f"First element shape: {lookup_array.shape}")
        else:
            raise ValueError("lookup_tables_3D_to_4D list is empty")
    else:
        lookup_array = lookup_table
        print(f"lookup_tables_3D_to_4D shape: {lookup_array.shape}")
    
    # Handle different possible shapes
    if lookup_array.ndim == 1:
        # If 1D, assume it's a single trial's time indices
        trial_n_timestamps = len(lookup_array)
        nTrials = 1
        print(f"Detected 1D lookup table: {trial_n_timestamps} timestamps, 1 trial")
    elif lookup_array.ndim == 2:
        trial_n_timestamps = lookup_array.shape[1]
        nTrials = lookup_array.shape[0]
        print(f"Detected 2D lookup table: {trial_n_timestamps} timestamps, {nTrials} trials")
    else:
        raise ValueError(f"Unexpected lookup table dimensions: {lookup_array.shape}")
    
    yPix, xPix, nWindows = iDopP.shape
    
    # create doppler struct that is (yPix, xPix, nWindows, nTrials) aligned to event
    train_data = np.full((yPix, xPix, trial_n_timestamps, nTrials), np.nan)
    
    for tt in range(nTrials):
        # Access lookup tables based on loading method
        if isinstance(dop_trials_struct, dict):
            lookup_tables = dop_trials_struct['dop_trials_struct'].get('lookup_tables_3D_to_4D', [])
            if isinstance(lookup_tables, list) and len(lookup_tables) > 0:
                timeidx = lookup_tables[0][:, tt] - 1  # Convert to 0-based indexing
            else:
                # Create default time indices if lookup tables not available
                timeidx = np.arange(trial_n_timestamps)
        else:
            timeidx = dop_trials_struct['dop_trials_struct']['lookup_tables_3D_to_4D'][0,0][0][:, tt] - 1  # Convert to 0-based indexing
        
        valid_idx = ~np.isnan(timeidx)
        if np.any(valid_idx):
            timeidx = timeidx[valid_idx].astype(int)
            # Make sure indices are within bounds
            timeidx = timeidx[timeidx < nWindows]
            if len(timeidx) > 0:
                train_data[:, :, valid_idx[:len(timeidx)], tt] = iDopP[:, :, timeidx]
    
    train_data = train_data.reshape(yPix * xPix, trial_n_timestamps, nTrials)
    train_data = train_data[:, :, keepidx]
    
    # decode across trial time
    nWindows = train_data.shape[1]
    
    # Determine dimensionality of cPCA subspace (if used)
    # For now, using (number of classes - 1), i.e m = 2 for 3 classes
    m = len(np.unique(train_labels_horz)) - 1
    
    # allocate memory fixation position
    result_horz = [None] * nWindows
    confusion_horz = [None] * nWindows
    cp_horz = [None] * nWindows
    class_predicted_horz = [None] * nWindows
    p_horz = np.full(nWindows, np.nan)
    percentCorrect_horz = np.full(nWindows, np.nan)
    t_horz = np.full(nWindows, np.nan)
    
    result_vert = [None] * nWindows
    confusion_vert = [None] * nWindows
    cp_vert = [None] * nWindows
    class_predicted_vert = [None] * nWindows
    p_vert = np.full(nWindows, np.nan)
    percentCorrect_vert = np.full(nWindows, np.nan)
    t_vert = np.full(nWindows, np.nan)
    
    downsample_interval = 1  # in seconds. average sequential frames within this interval together
    decoding_time_stepsize = 1  # in seconds. instead of decoding every frame, step forward this many second with each loop
    
    # set decoding start and stop time
    decoding_start_time = decoding_time_stepsize  # earliest start
    decoding_end_time = nWindows  # latest stop
    
    # Access acquisition rate based on loading method
    if isinstance(dop_trials_struct, dict):
        acquisition_rate = dop_trials_struct['dop_trials_struct'].get('acquisition_rate', 1.0)
        if isinstance(acquisition_rate, (list, np.ndarray)):
            acquisition_rate = acquisition_rate[0] if len(acquisition_rate) > 0 else 1.0
    else:
        acquisition_rate = dop_trials_struct['dop_trials_struct']['acquisition_rate'][0,0][0,0]
    
    if (downsample_interval * acquisition_rate) % 1 != 0:
        raise ValueError('error: the downsample_interval must correspond to an integer number of frames')
    if (decoding_time_stepsize * acquisition_rate) % 1 != 0:
        raise ValueError('error: the decoding_time_stepsize must correspond to an integer number of frames')
    if (trailingwindow * acquisition_rate) % 1 != 0:
        raise ValueError('error: the trailingwindow must correspond to an integer number of frames')
    
    trailingwindow = int(trailingwindow * acquisition_rate)  # TC. timepoints behind current that are included in decoder
    
    # Access trial time vector based on loading method
    if isinstance(dop_trials_struct, dict):
        trial_timevec = dop_trials_struct['dop_trials_struct'].get('trial_timevec', [np.arange(trial_n_timestamps)])
        if isinstance(trial_timevec, list) and len(trial_timevec) > 0:
            trialTime = np.array(trial_timevec[0]).flatten()
        else:
            trialTime = np.arange(trial_n_timestamps)
    else:
        trialTime = dop_trials_struct['dop_trials_struct']['trial_timevec'][0,0][0].flatten()
    
    print('Time with for loop')
    variance_to_keep = 95
    
    # want to keep 0.985 percent of variance
    # average frames in last 5 seconds
    for ii in range(int(decoding_start_time * acquisition_rate), 
                    int(decoding_end_time), 
                    int(decoding_time_stepsize * acquisition_rate)):
        
        testTimeI = np.arange(max(0, ii - trailingwindow), ii + 1)
        
        # prepare the training data for this time window, down-sampled
        trainData = []
        if len(testTimeI) < (downsample_interval * acquisition_rate):
            # Not enough data — use mean over available
            downsample_data = np.mean(train_data[:, testTimeI, :], axis=1)
            trainData.append(flattenDoppler2D(downsample_data, 1))
        else:
            # Main downsampling loop
       
        # if len(testTimeI) < (downsample_interval * acquisition_rate):
        #     downsample_data = np.mean(train_data[:, testTimeI, :], axis=1)
        #     trainData.append(flattenDoppler2D(downsample_data, axis=1))
        # else:
            # for jj in range(int(testTimeI[0] + downsample_interval * acquisition_rate - 1), 
            #                int(testTimeI[-1] + 1), 
            #                int(downsample_interval * acquisition_rate)):
            #     downsample_idx = np.arange(int(jj - downsample_interval * acquisition_rate), jj)
            #     downsample_data = np.mean(train_data[:, downsample_idx, :], axis=1)
            #     trainData.append(flattenDoppler2D(downsample_data, axis=1))
            for jj in range(testTimeI[0] + int(downsample_interval * acquisition_rate) - 1,
                        testTimeI[-1] + 1,
                        int(downsample_interval * acquisition_rate)):
                downsample_idx = list(range(jj - int(downsample_interval * acquisition_rate) + 1, jj + 1))
                # downsample_data = np.mean(train_data[:, downsample_idx, :], axis=1)

                # # Input to flattenDoppler2D is (yPix, xPix, time, nTrials) — emulate it:
                # yPix_xPix = int(np.sqrt(train_data.shape[0]))
                # downsample_data_reshaped = downsample_data.reshape((yPix_xPix, yPix_xPix, 1, train_data.shape[2]))

                downsample_data = np.mean(train_data[:, testTimeI, :], axis=1)  # shape: (yPix * xPix, nTrials)

                # Reshape properly using original yPix and xPix:
                downsample_data_reshaped = downsample_data.reshape((yPix, xPix, 1, train_data.shape[2]))

                # Now call flattenDoppler2D
                trainData.append(flattenDoppler2D(downsample_data_reshaped, 0))

            
        
        if trainData:
            trainData = np.concatenate(trainData, axis=1) if len(trainData) > 1 else trainData[0]
        
        if len(axes_to_decode) == 0 or axes_to_decode == 1:
            cp_horz[ii], p_horz[ii], class_predicted_horz[ii] = crossvalidate_short(
                trainData, train_labels_horz,
                classificationMethod=classifierString, N=N,
                validationMethod=validationString, K=K,
                m=m, variance_to_keep=variance_to_keep)
            
            percentCorrect_horz[ii] = cp_horz[ii]['CorrectRate'] * 100
            confusion_horz[ii] = cp_horz[ii]['CountingMatrix']
            
            best_horz_idx = np.nanargmin(p_horz)
        
        if len(axes_to_decode) == 0 or axes_to_decode == 2:
            cp_vert[ii], p_vert[ii], class_predicted_vert[ii] = crossvalidate_short(
                trainData, train_labels_vert,
                classificationMethod=classifierString, N=N,
                validationMethod=validationString, K=K,
                m=m, variance_to_keep=variance_to_keep)
            
            percentCorrect_vert[ii] = cp_vert[ii]['CorrectRate'] * 100
            confusion_vert[ii] = cp_vert[ii]['CountingMatrix']
        
        # save the data to results
        result_horz[ii] = {
            'percentCorrect': percentCorrect_horz[ii],
            'confusion': confusion_horz[ii],
            'p': p_horz[ii],
            't': ii - 1
        }
        
        result_vert[ii] = {
            'percentCorrect': percentCorrect_vert[ii],
            'confusion': confusion_vert[ii],
            'p': p_vert[ii],
            't': ii - 1
        }
        
        print(f"Processing timepoint: {ii}")
    
    downsample_check_struct = result_horz
    
    # remove the missing timepoints due to time_stepsize downsampling
    elim_idx_downsampling = np.zeros(len(downsample_check_struct), dtype=bool)
    for ii in range(len(downsample_check_struct)):
        if downsample_check_struct[ii] is None:
            elim_idx_downsampling[ii] = True
    
    # Remove eliminated indices
    result_horz = [result_horz[i] for i in range(len(result_horz)) if not elim_idx_downsampling[i]]
    result_vert = [result_vert[i] for i in range(len(result_vert)) if not elim_idx_downsampling[i]]
    
    percentCorrect_horz = percentCorrect_horz[~elim_idx_downsampling]
    confusion_horz = [confusion_horz[i] for i in range(len(confusion_horz)) if not elim_idx_downsampling[i]]
    p_horz = p_horz[~elim_idx_downsampling]
    class_predicted_horz = [class_predicted_horz[i] for i in range(len(class_predicted_horz)) if not elim_idx_downsampling[i]]
    
    percentCorrect_vert = percentCorrect_vert[~elim_idx_downsampling]
    confusion_vert = [confusion_vert[i] for i in range(len(confusion_vert)) if not elim_idx_downsampling[i]]
    p_vert = p_vert[~elim_idx_downsampling]
    class_predicted_vert = [class_predicted_vert[i] for i in range(len(class_predicted_vert)) if not elim_idx_downsampling[i]]
    
    trialTime = trialTime[~elim_idx_downsampling]
    
    # create decoding results struct
    if 'label_horz' not in decodingResults:
        decodingResults['label_horz'] = []
        decodingResults['label_vert'] = []
        decodingResults['keepidx'] = []
        decodingResults['train_labels_horz'] = []
        decodingResults['train_labels_vert'] = []
        decodingResults['result_horz'] = []
        decodingResults['result_vert'] = []
        decodingResults['percentCorrect_horz'] = []
        decodingResults['confusion_horz'] = []
        decodingResults['p_horz'] = []
        decodingResults['class_predicted_horz'] = []
        decodingResults['percentCorrect_vert'] = []
        decodingResults['confusion_vert'] = []
        decodingResults['p_vert'] = []
        decodingResults['class_predicted_vert'] = []
        decodingResults['trialTime'] = []
    
    decodingResults['label_horz'].append(label_horz)
    decodingResults['label_vert'].append(label_vert)
    decodingResults['keepidx'].append(keepidx)
    decodingResults['train_labels_horz'].append(train_labels_horz)
    decodingResults['train_labels_vert'].append(train_labels_vert)
    decodingResults['result_horz'].append(result_horz)
    decodingResults['result_vert'].append(result_vert)
    decodingResults['percentCorrect_horz'].append(percentCorrect_horz)
    decodingResults['confusion_horz'].append(confusion_horz)
    decodingResults['p_horz'].append(p_horz)
    decodingResults['class_predicted_horz'].append(class_predicted_horz)
    decodingResults['percentCorrect_vert'].append(percentCorrect_vert)
    decodingResults['confusion_vert'].append(confusion_vert)
    decodingResults['p_vert'].append(p_vert)
    decodingResults['class_predicted_vert'].append(class_predicted_vert)
    decodingResults['trialTime'].append(trialTime)
    
    print(f'done iter {nn + 1}')

print('finished decoding!')

## create variables from previously saved decoding_results struct, for plotting

percentCorrect_horz = np.array(decodingResults['percentCorrect_horz']).flatten()
percentCorrect_vert = np.array(decodingResults['percentCorrect_vert']).flatten()

p_horz = np.array(decodingResults['p_horz']).flatten()
p_vert = np.array(decodingResults['p_vert']).flatten()

train_labels_horz = np.array(decodingResults['train_labels_horz']).flatten()
train_labels_vert = np.array(decodingResults['train_labels_vert']).flatten()

trialTime = np.array(decodingResults['trialTime']).flatten()

# redo this so can handle multiple iters
confusion_horz = decodingResults['confusion_horz'][0]
confusion_vert = decodingResults['confusion_vert'][0]

## plot results direction

p_threshold = 0.05
xdat = trialTime

plt.figure(figsize=(10, 8))

# Horizontal direction
ydat = np.mean(percentCorrect_horz.reshape(1, -1), axis=0)
pdat = np.mean(p_horz.reshape(1, -1), axis=0)
chance_level = 100 / len(np.unique(train_labels_horz))
bestidx_horz = np.argmax(ydat)

plt.plot(xdat, ydat, linewidth=2, color='k', label='horizontal direction')

# plot p value & labels
significant_points = pdat < (p_threshold / len(xdat))
if np.any(significant_points):
    plt.plot(xdat[significant_points], np.full(np.sum(significant_points), 99), '*k', markersize=6)

plt.axhline(y=chance_level, color='k', linestyle='--')
plt.text(15, 20, 'horizontal direction', color='k', fontsize=11)

# Vertical direction
ydat_vert = np.mean(percentCorrect_vert.reshape(1, -1), axis=0)
pdat_vert = np.mean(p_vert.reshape(1, -1), axis=0)
chance_level_vert = 100 / len(np.unique(train_labels_vert))

if chance_level_vert < 100:
    bestidx_vert = np.argmax(ydat_vert)
    plt.plot(xdat, ydat_vert, linewidth=2, color='b', label='vertical direction')
    
    # plot p value & labels
    significant_points_vert = pdat_vert < (p_threshold / len(xdat))
    if np.any(significant_points_vert):
        plt.plot(xdat[significant_points_vert], np.full(np.sum(significant_points_vert), 98), '*b', markersize=6)
    
    plt.axhline(y=chance_level_vert, color='b', linestyle='--')
    plt.text(15, 16, 'vertical direction', color='b', fontsize=11)

plt.xlabel('time (s)')
plt.ylabel('% correct')
plt.title(f"session {decodingResults['session']}")
plt.xlim([xdat[0], xdat[-1]])
plt.ylim([0, 100])
plt.gca().set_aspect('equal')

plt.axvline(x=0, color='b')
plt.text(0.3, 80, 'Cue', color='b', fontsize=11)

plt.tick_params(labelsize=15)
plt.show()

# Uncomment to save results
# savetime = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
# savename = f"decodingResults_{protocol_name}_S{decodingResults['session']}_{savetime}_{processing}"
# savemat(savename + '.mat', {'decodingResults': decodingResults})

Successfully loaded MATLAB v7.3 file with h5py
=== Data Structure Debug ===
#refs#: dict with keys ['0', '00', '00b', '00c', '00d', '01', '01b', '01c', '01d', '02', '02b', '02c', '02d', '03', '03b', '03c', '03d', '04', '04b', '04c', '04d', '05', '05b', '05c', '05d', '06', '06b', '06c', '06d', '07', '07b', '07c', '07d', '08', '08b', '08c', '08d', '09', '09b', '09c', '09d', '0A', '0Ab', '0Ac', '0Ad', '0Ae', '0B', '0Bb', '0Bc', '0Bd', '0Be', '0C', '0Cb', '0Cc', '0Cd', '0Ce', '0D', '0Db', '0Dc', '0Dd', '0De', '0E', '0Eb', '0Ec', '0Ed', '0Ee', '0F', '0Fb', '0Fc', '0Fd', '0Fe', '0G', '0Gb', '0Gc', '0Gd', '0Ge', '0H', '0Hb', '0Hc', '0Hd', '0He', '0I', '0Ib', '0Ic', '0Id', '0Ie', '0J', '0Jb', '0Jc', '0Jd', '0Je', '0K', '0Kb', '0Kc', '0Kd', '0Ke', '0L', '0Lb', '0Lc', '0Ld', '0Le', '0M', '0Mb', '0Mc', '0Md', '0Me', '0N', '0Nb', '0Nc', '0Nd', '0Ne', '0O', '0Ob', '0Oc', '0Od', '0P', '0Pb', '0Pc', '0Pd', '0Q', '0Qb', '0Qc', '0Qd', '0R', '0Rb', '0Rc', '0Rd', '0S', '0Sb', '0Sc', '0Sd', '0T', '0Tb', '

MemoryError: Unable to allocate 175. MiB for an array with shape (17024, 10800) and data type bool

In [8]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.impute import SimpleImputer

def classifydoppler_short(train_data, train_labels, test_data, **kwargs):
    """
    Classifies doppler data using the training data and labels provided
    
    Parameters:
    -----------
    train_data : array_like
        Training data, size (nImagesL&R, nPixels*yPixels)
    train_labels : array_like
        Training labels, size (nImagesL&R,)
    test_data : array_like
        Testing data of size (nImagesToTest, nPixels*yPixels)
    
    Keyword Arguments:
    ------------------
    m : int, optional
        Scalar parameter (default: 1)
    method : str, optional
        Method type. Accepted values: 'PCA+LDA' (default: 'PCA+LDA')
    model : dict or None, optional
        Pre-trained model (default: None)
    variance_to_keep : float, optional
        Percentage of variance to keep for PCA (default: 95)
    handle_nan : str, optional
        How to handle NaN values: 'impute' (default), 'remove', or 'error'
        
    Returns:
    --------
    pred : ndarray
        Predictions for test data
    model : dict
        Trained model containing PCA and LDA components
    """
    
    # Parse keyword arguments
    m = kwargs.get('m', 1)
    method = kwargs.get('method', 'PCA+LDA')
    model = kwargs.get('model', None)
    variance_to_keep = kwargs.get('variance_to_keep', 95)
    handle_nan = kwargs.get('handle_nan', 'impute')
    
    # Convert to numpy arrays
    train_data = np.array(train_data, dtype=np.float64)
    train_labels = np.array(train_labels)
    test_data = np.array(test_data, dtype=np.float64)
    
    # Handle NaN values in labels first
    nan_label_indices = np.isnan(train_labels)
    if np.any(nan_label_indices):
        print(f"Removing {np.sum(nan_label_indices)} samples with NaN labels")
        train_labels = train_labels[~nan_label_indices]
        train_data = train_data[~nan_label_indices]
    
    # Handle NaN values in data
    train_has_nan = np.isnan(train_data).any()
    test_has_nan = np.isnan(test_data).any()
    
    if train_has_nan or test_has_nan:
        print(f"NaN values detected - Train: {np.isnan(train_data).sum()}, Test: {np.isnan(test_data).sum()}")
        
        if handle_nan == 'error':
            raise ValueError("NaN values found in data and handle_nan='error'")
        
        elif handle_nan == 'remove':
            # Remove samples with any NaN values
            valid_train_mask = ~np.isnan(train_data).any(axis=1)
            train_data = train_data[valid_train_mask]
            train_labels = train_labels[valid_train_mask]
            
            valid_test_mask = ~np.isnan(test_data).any(axis=1)
            test_data = test_data[valid_test_mask]
            
            print(f"Removed samples - Train: {np.sum(~valid_train_mask)}, Test: {np.sum(~valid_test_mask)}")
            
        elif handle_nan == 'impute':
            # Impute missing values
            if model is None or 'imputer' not in model:
                # Fit imputer on training data
                imputer = SimpleImputer(strategy='mean')
                train_data = imputer.fit_transform(train_data)
                test_data = imputer.transform(test_data)
                store_imputer = True
            else:
                # Use existing imputer
                imputer = model['imputer']
                train_data = imputer.transform(train_data)
                test_data = imputer.transform(test_data)
                store_imputer = False
        else:
            raise ValueError(f"Unknown handle_nan method: {handle_nan}")
    
    # Check if we have enough samples after cleaning
    if len(train_data) == 0:
        raise ValueError("No valid training samples remaining after NaN handling")
    
    # Check if we have at least 2 classes
    unique_labels = np.unique(train_labels)
    if len(unique_labels) < 2:
        raise ValueError(f"Need at least 2 classes for classification, found {len(unique_labels)}")
    
    # PCA+LDA method
    if method == 'PCA+LDA':
        if model is None or not isinstance(model, dict):
            # Train new model
            
            # PCA
            pca = PCA()
            pca_scores = pca.fit_transform(train_data)
            pca_coefficients = pca.components_.T  # Transpose to match MATLAB convention
            explained_variance_ratio = pca.explained_variance_ratio_
            pca_centers = pca.mean_
            
            # Determine number of components to keep
            explained_variance_to_keep_as_fraction = variance_to_keep / 100
            
            if variance_to_keep < 100:
                cumsum_explained = np.cumsum(explained_variance_ratio)
                num_components_to_keep = np.where(cumsum_explained >= explained_variance_to_keep_as_fraction)[0]
                
                if len(num_components_to_keep) == 0:
                    # If we can't reach the desired variance, use all components
                    print(f"Warning: Could not reach {variance_to_keep}% variance. Using all {len(explained_variance_ratio)} components.")
                    num_components_to_keep = len(explained_variance_ratio)
                else:
                    num_components_to_keep = num_components_to_keep[0] + 1
                
                pca_coefficients = pca_coefficients[:, :num_components_to_keep]
                train_predictors = pca_scores[:, :num_components_to_keep]
            else:
                train_predictors = pca_scores
            
            # LDA
            mdl_linear = LinearDiscriminantAnalysis()
            mdl_linear.fit(train_predictors, train_labels)
            
            # Create model structure
            model = {
                'pca_centers': pca_centers,
                'pca_coefficients': pca_coefficients,
                'mdl_linear': mdl_linear
            }
            
            # Add imputer to model if we used one
            if handle_nan == 'impute' and store_imputer:
                model['imputer'] = imputer
                
        else:
            # Use existing model
            pca_centers = model['pca_centers']
            pca_coefficients = model['pca_coefficients']
            mdl_linear = model['mdl_linear']
        
        # Transform test data using PCA
        test_data_pca = (test_data - pca_centers) @ pca_coefficients
        
        # Make predictions
        pred = mdl_linear.predict(test_data_pca)
    
    else:
        raise ValueError(f"Unknown method: {method}. Only 'PCA+LDA' is supported.")
    
    return pred, model

In [9]:
import numpy as np
from scipy.stats import zscore
from sklearn.model_selection import KFold
from scipy.stats import binom
from binomialTest import binomial_test


def crossvalidate_short(data, labels, verbose=False, validationMethod='kFold', K=10, N=30,
                       classificationMethod='PCA+LDA', testData=None, m=1, variance_to_keep=95,
                       trial_ind=None):
    """
    crossValidate will load data from a set of session/run combinations
    and then cross validate those results using your method of choice
    
    Parameters:
    data: array of shape (nImagesL&R x nPixels*yPixels)
    labels: array of shape (nImages,) with values like [0, 1, 0, 0, 1, 1, ...]
    verbose: boolean - print results
    validationMethod: string - options are 'kFold'
    K: integer - for K-fold validation (default: 10)
    N: integer - parameter for validation (default: 30)
    classificationMethod: string - options are 'PCA+LDA'
    testData: array of same shape as data - optional test data from different time window
    m: positive integer - subspace dimensions for cPCA (max: # classes - 1, default: 1)
    variance_to_keep: float - variance to keep in PCA (default: 95)
    trial_ind: array - trial indices (default: all trials)
    
    Returns:
    cp: dictionary containing classification performance metrics
    p: p-value from binomial test
    class_tracker: array of predicted classes for each trial
    """
    
    # Handle default trial indices
    if trial_ind is None:
        trial_ind = np.arange(len(labels))
    
    # Normalize to z-score
    data = zscore(data, axis=0)  # standardize features
    if testData is not None:
        testData = zscore(testData, axis=0)
    
    # Get useful variables
    N_trials = data.shape[0]  # number of trials
    
    # Initialize class performance tracking
    cp = {
        'CorrectRate': 0.0,
        'CountingMatrix': np.zeros((len(np.unique(labels)), len(np.unique(labels)))),
        'ErrorRate': 0.0,
        'LastClassName': None,
        'ClassNames': np.unique(labels),
        'TestTargets': [],
        'TestResults': []
    }
    
    # k-fold cross validation
    if validationMethod == 'kFold':
        # Create k-fold indices
        kfold = KFold(n_splits=K, shuffle=True, random_state=42)
        class_tracker = np.full(N_trials, np.nan)
        
        fold_idx = 0
        # For each k-fold
        for train_idx, test_idx in kfold.split(data):
            fold_idx += 1
            
            # Get training and test data
            train_data = data[train_idx]
            train_labels = labels[train_idx]
            
            if testData is not None:
                test_data = testData[test_idx]
            else:
                test_data = data[test_idx]
            
            test_labels = labels[test_idx]
            
            # Classify using your classification function
            # Note: You need to implement classifyDoppler_short
            predicted_classes, model = classifydoppler_short(
                train_data, train_labels, test_data,
                method=classificationMethod,
                m=m,
                variance_to_keep=variance_to_keep
            )
            
            # Update class performance
            classperf_update(cp, predicted_classes, test_labels)
            class_tracker[test_idx] = predicted_classes
            
            # Optional progress tracking
            # print(f'Finished fold {fold_idx}/{K}')
    
    # Calculate classification accuracy measures
    nCorrect = np.sum(np.diag(cp['CountingMatrix']))
    nCounted = np.sum(cp['CountingMatrix'])
    
    if nCounted > 0:
        cp['CorrectRate'] = nCorrect / nCounted
        cp['ErrorRate'] = 1 - cp['CorrectRate']
    
    percentCorrect = cp['CorrectRate'] * 100
    chance = 1 / len(np.unique(labels))
    
    # Binomial test for significance
    p = binomial_test(nCorrect, nCounted, chance, 'one')
    
    # Display measures if verbose is on
    if verbose:
        print(f'\nClassification Accuracy:')
        print(f'{int(nCorrect)} / {int(nCounted)} trials correctly classified ({percentCorrect:.2f}% correct)\t', end='')
        
        if p < 0.001:
            print('(binomial test: p < 0.001)')
        else:
            print(f'(binomial test: p = {p:.3f})')
    
    return cp, p, class_tracker


def classperf_update(cp, predicted_classes, true_labels):
    """
    Update classification performance metrics
    Equivalent to MATLAB's classperf function
    """
    # Convert to numpy arrays
    predicted_classes = np.array(predicted_classes).flatten()
    true_labels = np.array(true_labels)
    
    # Store test results
    cp['TestTargets'].extend(true_labels.tolist())
    cp['TestResults'].extend(predicted_classes.tolist())
    
    # Update confusion matrix
    unique_labels = cp['ClassNames']
    for i, true_label in enumerate(unique_labels):
        for j, pred_label in enumerate(unique_labels):
            mask_true = (true_labels == true_label)
            mask_pred = (predicted_classes == pred_label)
            cp['CountingMatrix'][i, j] += np.sum(mask_true & mask_pred)

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
from datetime import datetime
import os
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from detrend_sliding_window import detrend_sliding_window
# from crossvalidate_short import crossvalidate_short
from preprocess import preProcess
from flattenDoppler2D import flattenDoppler2D
import warnings
warnings.filterwarnings('ignore')
import h5py


# TC: for early direction/value tests with Verasonics, Sessions [164 165 166] - 4 corners 2 values
# simplified script - for Bahareh 20240606

# load a dop_trials_struct before proceeding
def load_matlab_v73(filepath):
    """
    Load MATLAB v7.3 files using h5py
    """
    def h5py_to_dict(h5_file, path='/'):
        """Recursively convert h5py file to dictionary"""
        data = {}
        for key in h5_file[path].keys():
            if isinstance(h5_file[path + key], h5py.Dataset):
                # Handle different data types
                dataset = h5_file[path + key]
                if dataset.dtype.kind == 'O':  # Object references
                    # Handle cell arrays or struct arrays
                    try:
                        # Try to dereference object references
                        refs = dataset[:]
                        if refs.ndim == 2 and refs.shape[0] == 1:
                            refs = refs[0]
                        
                        deref_data = []
                        for ref in refs.flat:
                            if ref:
                                deref_data.append(h5_file[ref][:])
                        
                        if len(deref_data) == 1:
                            data[key] = deref_data[0]
                        else:
                            data[key] = deref_data
                    except:
                        # If dereferencing fails, just get the raw data
                        data[key] = dataset[:]
                else:
                    # Regular numeric data
                    arr = dataset[:]
                    # MATLAB stores arrays in column-major order, transpose if needed
                    if arr.ndim > 1:
                        data[key] = arr.T
                    else:
                        data[key] = arr
            elif isinstance(h5_file[path + key], h5py.Group):
                # Recursively handle groups (structs)
                data[key] = h5py_to_dict(h5_file, path + key + '/')
        return data
    
    with h5py.File(filepath, 'r') as f:
        return h5py_to_dict(f)

def debug_data_structure(data_dict, max_depth=3, current_depth=0):
    """Debug function to examine the structure of loaded data"""
    if current_depth > max_depth:
        return
    
    for key, value in data_dict.items():
        indent = "  " * current_depth
        if isinstance(value, dict):
            print(f"{indent}{key}: dict with keys {list(value.keys())}")
            debug_data_structure(value, max_depth, current_depth + 1)
        elif isinstance(value, np.ndarray):
            print(f"{indent}{key}: array shape {value.shape}, dtype {value.dtype}")
        elif isinstance(value, list):
            print(f"{indent}{key}: list of length {len(value)}")
            if len(value) > 0 and isinstance(value[0], np.ndarray):
                print(f"{indent}  First element shape: {value[0].shape}")
        else:
            print(f"{indent}{key}: {type(value)}")

# Neural decoding analysis script - Python conversion of MATLAB code
try:
    # First try with h5py for MATLAB v7.3 files
    dop_trials_struct = load_matlab_v73(r"D:\Downloads\dop_trials_struct_S164_R1_NoRegister_NoDetrend.mat")
    print("Successfully loaded MATLAB v7.3 file with h5py")
except Exception as e:
    print(f"h5py loading failed: {e}")
    try:
        # Fallback to scipy.io.loadmat for older MATLAB files
        from scipy.io import loadmat
        dop_trials_struct = loadmat(r"D:\Downloads\dop_trials_struct_S164_R1_NoRegister_NoDetrend.mat")
        print("Successfully loaded with scipy.io.loadmat")
    except Exception as e2:
        print(f"Both loading methods failed. h5py error: {e}, scipy error: {e2}")
        raise

# Debug the data structure
print("=== Data Structure Debug ===")
debug_data_structure(dop_trials_struct)

processing = 'NoRegister_NoDetrend'

# choose previous N trials for adaptive training set
N = 175
# choose K for K-fold validation
K = 15
# decoding window in seconds
trailingwindow = 4

# control for preprocessing steps

# Fits an exponential model to the mean signal across depth and uses the
# model to normalize the data. Makes sure that signals from deeper regions
# are properly scaled.
timeGain = False

# Averaging filter that smooths each 2D slice (maps to each window-trial)
# from input data
diskFilter = False

# zscore is applied to every voxel.
zScore = True

# Removes low frequency trends (tissue movement/environment influences)
# from the Doppler data using a sliding window approach of 50 seconds.
detrend = False

# Question: Are windows overlapping?
# Clarify: We are averaging?
if detrend:
    detrend_window_length = 50  # seconds
    iDopP = detrend_sliding_window(dop_trials_struct['dop_trials_struct']['iDopP'], 
                                  detrend_window_length * dop_trials_struct['dop_trials_struct']['acquisition_rate'])
else:
    detrend_window_length = 0
    iDopP = dop_trials_struct['dop_trials_struct']['iDopP']
    
# if detrend:
#     detrend_window_length = 50  # seconds
#     # Access data based on whether it's h5py loaded or scipy loaded
#     if isinstance(dop_trials_struct, dict) and 'iDopP' in dop_trials_struct:
#         iDopP_data = dop_trials_struct['dop_trials_struct']['iDopP']
#         acq_rate = dop_trials_struct['dop_trials_struct']['acquisition_rate']
#     else:
#         # Fallback for scipy.io loaded data
#         iDopP_data = dop_trials_struct['dop_trials_struct']['iDopP'][0,0]
#         acq_rate = dop_trials_struct['dop_trials_struct']['acquisition_rate'][0,0][0,0]
    
#     iDopP = detrend_sliding_window(iDopP_data, detrend_window_length * acq_rate)
# else:
#     detrend_window_length = 0
#     # Access data based on loading method
#     if isinstance(dop_trials_struct, dict) and 'iDopP' in dop_trials_struct:
#         iDopP = dop_trials_struct['dop_trials_struct']['iDopP']
#     else:
#         iDopP = dop_trials_struct['dop_trials_struct']['iDopP'][0,0]

# Performs these preprocessing steps
print(f"iDopP shape before preProcess: {iDopP.shape}")
print(f"iDopP dtype: {iDopP.dtype}")
print(f"Contains NaN: {np.any(np.isnan(iDopP))}")
iDopP = preProcess(iDopP, timeGain=timeGain, diskFilter=diskFilter, zScore=zScore)

# No Register - this means no motion correction but yes detrend
# No Register + No Detrend - no motion correction and no detrend
# Normcorre - motion correction...no detrend (??)

# Now we do PCA + LDA.

# PCA does dimensionality reduction by transforming the
# original variables to new set of uncorrelated variables that capture most
# variance. PCA also does noise reduction as it focuses on the principal
# components that capture the most variance, and inform the patterns in
# brain signals

# LDA separates the classes in the best way possible as it finds the
# discriminants that maximizes the distance between classes

classifierString = 'PCA+LDA'
validationString = 'kFold'

protocol_name = 'ValDir'
n_iter = 1

decodingResults = {}
decodingResults['session'] = dop_trials_struct['dop_trials_struct'].get('session', 0)
decodingResults['run'] = dop_trials_struct['dop_trials_struct'].get('run', 0)
decodingResults['protocol_name'] = protocol_name

## indexing labels

# manually set
only_use_successful_trials = 1
axes_to_decode = []  # [] for both, 1 for horz, 2 for vert


fixation_pos = dop_trials_struct['dop_trials_struct']['fixation_pos']
target_cue_pos = np.squeeze(dop_trials_struct['dop_trials_struct']['target_pos_eachCue']).T
tprime_cue_pos = np.squeeze(dop_trials_struct['dop_trials_struct']['targetPrime_pos_eachCue']).T
    
# reached_min_state = dop_trials_struct['dop_trials_struct'].get('reached_minimum_state_idx', np.array([]))
# success_idx = dop_trials_struct['dop_trials_struct'].get('success_idx', np.array([]))
    
success_trial_idx = np.isin(dop_trials_struct['dop_trials_struct']['reached_minimum_state_idx'].flatten(),
                           dop_trials_struct['dop_trials_struct']['success_idx'].flatten())

decodingResults['only_use_successful_trials'] = only_use_successful_trials
decodingResults['axes_to_decode'] = axes_to_decode

for nn in range(n_iter):
    # for saccade direction ipsi/contra or up/down
    label_horz = np.sign(target_cue_pos[:, 0])
    label_vert = np.sign(target_cue_pos[:, 1])
    
    # set to min of 1
    label_horz = label_horz - np.min(label_horz) + 1
    label_vert = label_vert - np.min(label_vert) + 1
    
    if only_use_successful_trials:
        # FIX: Handle the indexing properly based on the actual shape of success_trial_idx
        if success_trial_idx.ndim == 1:
            # If success_trial_idx is 1D, use it directly
            label_horz[~success_trial_idx] = np.nan
            label_vert[~success_trial_idx] = np.nan
        elif success_trial_idx.ndim == 2:
            # If success_trial_idx is 2D, use the first column as originally intended
            label_horz[~success_trial_idx[:, 0]] = np.nan
            label_vert[~success_trial_idx[:, 0]] = np.nan
        else:
            print(f"Warning: Unexpected success_trial_idx shape: {success_trial_idx.shape}")
            # Use flattened version as fallback
            success_trial_idx_flat = success_trial_idx.flatten()
            if len(success_trial_idx_flat) == len(label_horz):
                label_horz[~success_trial_idx_flat] = np.nan
                label_vert[~success_trial_idx_flat] = np.nan
            else:
                print("Cannot match success_trial_idx to labels, skipping successful trials filter")


    # if only_use_successful_trials:
    #     label_horz[~success_trial_idx[:, 0]] = np.nan
    #     label_vert[~success_trial_idx[:, 0]] = np.nan
    
    # equalize counts of each label combination
    keepidx = []
    mincount_combo = 10000
    
    if len(axes_to_decode) == 0:
        unique_horz = np.unique(label_horz[~np.isnan(label_horz)])
        unique_vert = np.unique(label_vert[~np.isnan(label_vert)])
    elif axes_to_decode == 1:
        unique_horz = np.unique(label_horz[~np.isnan(label_horz)])
        unique_vert = [1]
    elif axes_to_decode == 2:
        unique_horz = [1]
        unique_vert = np.unique(label_vert[~np.isnan(label_vert)])
    
    # find minimum count of combos
    for u1 in range(len(unique_horz)):
        for u2 in range(len(unique_vert)):
            if len(axes_to_decode) == 0:
                idx = (label_horz == unique_horz[u1]) & (label_vert == unique_vert[u2])
            elif axes_to_decode == 1:
                idx = label_horz == unique_horz[u1]
            elif axes_to_decode == 2:
                idx = label_vert == unique_vert[u2]
            mincount_combo = min(mincount_combo, np.sum(idx))
    
    # create set with equal representation all combos
    for u1 in range(len(unique_horz)):
        for u2 in range(len(unique_vert)):
            if len(axes_to_decode) == 0:
                idx = (label_horz == unique_horz[u1]) & (label_vert == unique_vert[u2])
            elif axes_to_decode == 1:
                idx = label_horz == unique_horz[u1]
            elif axes_to_decode == 2:
                idx = label_vert == unique_vert[u2]
            
            idx_indices = np.where(idx)[0]
            selected_indices = np.random.choice(idx_indices, mincount_combo, replace=False)
            keepidx.extend(selected_indices)
    
    keepidx = np.sort(keepidx)
    
    train_labels_horz = label_horz[keepidx]
    train_labels_vert = label_vert[keepidx]
    
    print(f"train_labels_horz shape: {train_labels_horz.shape}")
    
    lookup_table = dop_trials_struct['dop_trials_struct']['lookup_tables_3D_to_4D']

    print(f"lookup_tables_3D_to_4D type: {type(lookup_table)}")
    if isinstance(lookup_table, list):
        print(f"lookup_tables_3D_to_4D is a list of length: {len(lookup_table)}")
        if len(lookup_table) > 0:
            lookup_array = lookup_table[0]
            print(f"First element shape: {lookup_array.shape}")
        else:
            raise ValueError("lookup_tables_3D_to_4D list is empty")
    else:
        lookup_array = lookup_table
        print(f"lookup_tables_3D_to_4D shape: {lookup_array.shape}")
    
    # Handle different possible shapes
    if lookup_array.ndim == 1:
        # If 1D, assume it's a single trial's time indices
        trial_n_timestamps = len(lookup_array)
        nTrials = 1
        print(f"Detected 1D lookup table: {trial_n_timestamps} timestamps, 1 trial")
    elif lookup_array.ndim == 2:
        trial_n_timestamps = lookup_array.shape[1]
        nTrials = lookup_array.shape[0]
        print(f"Detected 2D lookup table: {trial_n_timestamps} timestamps, {nTrials} trials")
    else:
        raise ValueError(f"Unexpected lookup table dimensions: {lookup_array.shape}")
    
    yPix, xPix, nWindows = iDopP.shape
    
    # create doppler struct that is (yPix, xPix, nWindows, nTrials) aligned to event
    train_data = np.full((yPix, xPix, trial_n_timestamps, nTrials), np.nan)
    
    for tt in range(nTrials):
        # Access lookup tables based on loading method
        if isinstance(dop_trials_struct, dict):
            lookup_tables = dop_trials_struct['dop_trials_struct'].get('lookup_tables_3D_to_4D', [])
            if isinstance(lookup_tables, list) and len(lookup_tables) > 0:
                timeidx = lookup_tables[0][:, tt] - 1  # Convert to 0-based indexing
            else:
                # Create default time indices if lookup tables not available
                timeidx = np.arange(trial_n_timestamps)
        else:
            timeidx = dop_trials_struct['dop_trials_struct']['lookup_tables_3D_to_4D'][0,0][0][:, tt] - 1  # Convert to 0-based indexing
        
        valid_idx = ~np.isnan(timeidx)
        if np.any(valid_idx):
            timeidx = timeidx[valid_idx].astype(int)
            # Make sure indices are within bounds
            timeidx = timeidx[timeidx < nWindows]
            if len(timeidx) > 0:
                train_data[:, :, valid_idx[:len(timeidx)], tt] = iDopP[:, :, timeidx]
    
    train_data = train_data.reshape(yPix * xPix, trial_n_timestamps, nTrials)
    train_data = train_data[:, :, keepidx]
    
    # decode across trial time
    nWindows = train_data.shape[1]
    
    # Determine dimensionality of cPCA subspace (if used)
    # For now, using (number of classes - 1), i.e m = 2 for 3 classes
    m = len(np.unique(train_labels_horz)) - 1
    
    # allocate memory fixation position
    result_horz = [None] * nWindows
    confusion_horz = [None] * nWindows
    cp_horz = [None] * nWindows
    class_predicted_horz = [None] * nWindows
    p_horz = np.full(nWindows, np.nan)
    percentCorrect_horz = np.full(nWindows, np.nan)
    t_horz = np.full(nWindows, np.nan)
    
    result_vert = [None] * nWindows
    confusion_vert = [None] * nWindows
    cp_vert = [None] * nWindows
    class_predicted_vert = [None] * nWindows
    p_vert = np.full(nWindows, np.nan)
    percentCorrect_vert = np.full(nWindows, np.nan)
    t_vert = np.full(nWindows, np.nan)
    
    downsample_interval = 1  # in seconds. average sequential frames within this interval together
    decoding_time_stepsize = 1  # in seconds. instead of decoding every frame, step forward this many second with each loop
    
    # set decoding start and stop time
    decoding_start_time = decoding_time_stepsize  # earliest start
    decoding_end_time = nWindows  # latest stop
    
    # Access acquisition rate based on loading method
    if isinstance(dop_trials_struct, dict):
        acquisition_rate = dop_trials_struct['dop_trials_struct'].get('acquisition_rate', 1.0)
        if isinstance(acquisition_rate, (list, np.ndarray)):
            acquisition_rate = acquisition_rate[0] if len(acquisition_rate) > 0 else 1.0
    else:
        acquisition_rate = dop_trials_struct['dop_trials_struct']['acquisition_rate'][0,0][0,0]
    
    if (downsample_interval * acquisition_rate) % 1 != 0:
        raise ValueError('error: the downsample_interval must correspond to an integer number of frames')
    if (decoding_time_stepsize * acquisition_rate) % 1 != 0:
        raise ValueError('error: the decoding_time_stepsize must correspond to an integer number of frames')
    if (trailingwindow * acquisition_rate) % 1 != 0:
        raise ValueError('error: the trailingwindow must correspond to an integer number of frames')
    
    trailingwindow = int(trailingwindow * acquisition_rate)  # TC. timepoints behind current that are included in decoder
    
    # Access trial time vector based on loading method
    if isinstance(dop_trials_struct, dict):
        trial_timevec = dop_trials_struct['dop_trials_struct'].get('trial_timevec', [np.arange(trial_n_timestamps)])
        if isinstance(trial_timevec, list) and len(trial_timevec) > 0:
            trialTime = np.array(trial_timevec[0]).flatten()
        else:
            trialTime = np.arange(trial_n_timestamps)
    else:
        trialTime = dop_trials_struct['dop_trials_struct']['trial_timevec'][0,0][0].flatten()
    
    print('Time with for loop')
    variance_to_keep = 95
    
    # want to keep 0.985 percent of variance
    # average frames in last 5 seconds
    for ii in range(int(decoding_start_time * acquisition_rate), 
                    int(decoding_end_time), 
                    int(decoding_time_stepsize * acquisition_rate)):
        
        testTimeI = np.arange(max(0, ii - trailingwindow), ii + 1)
        
        # prepare the training data for this time window, down-sampled
        trainData = []
        if len(testTimeI) < (downsample_interval * acquisition_rate):
            # Not enough data — use mean over available
            downsample_data = np.mean(train_data[:, testTimeI, :], axis=1)
            trainData.append(flattenDoppler2D(downsample_data, 1))
        else:
            # Main downsampling loop
       
        # if len(testTimeI) < (downsample_interval * acquisition_rate):
        #     downsample_data = np.mean(train_data[:, testTimeI, :], axis=1)
        #     trainData.append(flattenDoppler2D(downsample_data, axis=1))
        # else:
            # for jj in range(int(testTimeI[0] + downsample_interval * acquisition_rate - 1), 
            #                int(testTimeI[-1] + 1), 
            #                int(downsample_interval * acquisition_rate)):
            #     downsample_idx = np.arange(int(jj - downsample_interval * acquisition_rate), jj)
            #     downsample_data = np.mean(train_data[:, downsample_idx, :], axis=1)
            #     trainData.append(flattenDoppler2D(downsample_data, axis=1))
            for jj in range(testTimeI[0] + int(downsample_interval * acquisition_rate) - 1,
                        testTimeI[-1] + 1,
                        int(downsample_interval * acquisition_rate)):
                downsample_idx = list(range(jj - int(downsample_interval * acquisition_rate) + 1, jj + 1))
                # downsample_data = np.mean(train_data[:, downsample_idx, :], axis=1)

                # # Input to flattenDoppler2D is (yPix, xPix, time, nTrials) — emulate it:
                # yPix_xPix = int(np.sqrt(train_data.shape[0]))
                # downsample_data_reshaped = downsample_data.reshape((yPix_xPix, yPix_xPix, 1, train_data.shape[2]))

                downsample_data = np.mean(train_data[:, testTimeI, :], axis=1)  # shape: (yPix * xPix, nTrials)

                # Reshape properly using original yPix and xPix:
                downsample_data_reshaped = downsample_data.reshape((yPix, xPix, 1, train_data.shape[2]))

                # Now call flattenDoppler2D
                trainData.append(flattenDoppler2D(downsample_data_reshaped, 0))

            
        
        if trainData:
            trainData = np.concatenate(trainData, axis=1) if len(trainData) > 1 else trainData[0]
        
        if len(axes_to_decode) == 0 or axes_to_decode == 1:
            cp_horz[ii], p_horz[ii], class_predicted_horz[ii] = crossvalidate_short(
                trainData, train_labels_horz,
                classificationMethod=classifierString, N=N,
                validationMethod=validationString, K=K,
                m=m, variance_to_keep=variance_to_keep)
            
            percentCorrect_horz[ii] = cp_horz[ii]['CorrectRate'] * 100
            confusion_horz[ii] = cp_horz[ii]['CountingMatrix']
            
            best_horz_idx = np.nanargmin(p_horz)
        
        if len(axes_to_decode) == 0 or axes_to_decode == 2:
            cp_vert[ii], p_vert[ii], class_predicted_vert[ii] = crossvalidate_short(
                trainData, train_labels_vert,
                classificationMethod=classifierString, N=N,
                validationMethod=validationString, K=K,
                m=m, variance_to_keep=variance_to_keep)
            
            percentCorrect_vert[ii] = cp_vert[ii]['CorrectRate'] * 100
            confusion_vert[ii] = cp_vert[ii]['CountingMatrix']
        
        # save the data to results
        result_horz[ii] = {
            'percentCorrect': percentCorrect_horz[ii],
            'confusion': confusion_horz[ii],
            'p': p_horz[ii],
            't': ii - 1
        }
        
        result_vert[ii] = {
            'percentCorrect': percentCorrect_vert[ii],
            'confusion': confusion_vert[ii],
            'p': p_vert[ii],
            't': ii - 1
        }
        
        print(f"Processing timepoint: {ii}")
    
    downsample_check_struct = result_horz
    
    # remove the missing timepoints due to time_stepsize downsampling
    elim_idx_downsampling = np.zeros(len(downsample_check_struct), dtype=bool)
    for ii in range(len(downsample_check_struct)):
        if downsample_check_struct[ii] is None:
            elim_idx_downsampling[ii] = True
    
    # Remove eliminated indices
    result_horz = [result_horz[i] for i in range(len(result_horz)) if not elim_idx_downsampling[i]]
    result_vert = [result_vert[i] for i in range(len(result_vert)) if not elim_idx_downsampling[i]]
    
    percentCorrect_horz = percentCorrect_horz[~elim_idx_downsampling]
    confusion_horz = [confusion_horz[i] for i in range(len(confusion_horz)) if not elim_idx_downsampling[i]]
    p_horz = p_horz[~elim_idx_downsampling]
    class_predicted_horz = [class_predicted_horz[i] for i in range(len(class_predicted_horz)) if not elim_idx_downsampling[i]]
    
    percentCorrect_vert = percentCorrect_vert[~elim_idx_downsampling]
    confusion_vert = [confusion_vert[i] for i in range(len(confusion_vert)) if not elim_idx_downsampling[i]]
    p_vert = p_vert[~elim_idx_downsampling]
    class_predicted_vert = [class_predicted_vert[i] for i in range(len(class_predicted_vert)) if not elim_idx_downsampling[i]]
    
    trialTime = trialTime[~elim_idx_downsampling]
    
    # create decoding results struct
    if 'label_horz' not in decodingResults:
        decodingResults['label_horz'] = []
        decodingResults['label_vert'] = []
        decodingResults['keepidx'] = []
        decodingResults['train_labels_horz'] = []
        decodingResults['train_labels_vert'] = []
        decodingResults['result_horz'] = []
        decodingResults['result_vert'] = []
        decodingResults['percentCorrect_horz'] = []
        decodingResults['confusion_horz'] = []
        decodingResults['p_horz'] = []
        decodingResults['class_predicted_horz'] = []
        decodingResults['percentCorrect_vert'] = []
        decodingResults['confusion_vert'] = []
        decodingResults['p_vert'] = []
        decodingResults['class_predicted_vert'] = []
        decodingResults['trialTime'] = []
    
    decodingResults['label_horz'].append(label_horz)
    decodingResults['label_vert'].append(label_vert)
    decodingResults['keepidx'].append(keepidx)
    decodingResults['train_labels_horz'].append(train_labels_horz)
    decodingResults['train_labels_vert'].append(train_labels_vert)
    decodingResults['result_horz'].append(result_horz)
    decodingResults['result_vert'].append(result_vert)
    decodingResults['percentCorrect_horz'].append(percentCorrect_horz)
    decodingResults['confusion_horz'].append(confusion_horz)
    decodingResults['p_horz'].append(p_horz)
    decodingResults['class_predicted_horz'].append(class_predicted_horz)
    decodingResults['percentCorrect_vert'].append(percentCorrect_vert)
    decodingResults['confusion_vert'].append(confusion_vert)
    decodingResults['p_vert'].append(p_vert)
    decodingResults['class_predicted_vert'].append(class_predicted_vert)
    decodingResults['trialTime'].append(trialTime)
    
    print(f'done iter {nn + 1}')

print('finished decoding!')

## create variables from previously saved decoding_results struct, for plotting

percentCorrect_horz = np.array(decodingResults['percentCorrect_horz']).flatten()
percentCorrect_vert = np.array(decodingResults['percentCorrect_vert']).flatten()

p_horz = np.array(decodingResults['p_horz']).flatten()
p_vert = np.array(decodingResults['p_vert']).flatten()

train_labels_horz = np.array(decodingResults['train_labels_horz']).flatten()
train_labels_vert = np.array(decodingResults['train_labels_vert']).flatten()

trialTime = np.array(decodingResults['trialTime']).flatten()

# redo this so can handle multiple iters
confusion_horz = decodingResults['confusion_horz'][0]
confusion_vert = decodingResults['confusion_vert'][0]

## plot results direction

p_threshold = 0.05
xdat = trialTime

plt.figure(figsize=(10, 8))

# Horizontal direction
ydat = np.mean(percentCorrect_horz.reshape(1, -1), axis=0)
pdat = np.mean(p_horz.reshape(1, -1), axis=0)
chance_level = 100 / len(np.unique(train_labels_horz))
bestidx_horz = np.argmax(ydat)

plt.plot(xdat, ydat, linewidth=2, color='k', label='horizontal direction')

# plot p value & labels
significant_points = pdat < (p_threshold / len(xdat))
if np.any(significant_points):
    plt.plot(xdat[significant_points], np.full(np.sum(significant_points), 99), '*k', markersize=6)

plt.axhline(y=chance_level, color='k', linestyle='--')
plt.text(15, 20, 'horizontal direction', color='k', fontsize=11)

# Vertical direction
ydat_vert = np.mean(percentCorrect_vert.reshape(1, -1), axis=0)
pdat_vert = np.mean(p_vert.reshape(1, -1), axis=0)
chance_level_vert = 100 / len(np.unique(train_labels_vert))

if chance_level_vert < 100:
    bestidx_vert = np.argmax(ydat_vert)
    plt.plot(xdat, ydat_vert, linewidth=2, color='b', label='vertical direction')
    
    # plot p value & labels
    significant_points_vert = pdat_vert < (p_threshold / len(xdat))
    if np.any(significant_points_vert):
        plt.plot(xdat[significant_points_vert], np.full(np.sum(significant_points_vert), 98), '*b', markersize=6)
    
    plt.axhline(y=chance_level_vert, color='b', linestyle='--')
    plt.text(15, 16, 'vertical direction', color='b', fontsize=11)

plt.xlabel('time (s)')
plt.ylabel('% correct')
plt.title(f"session {decodingResults['session']}")
plt.xlim([xdat[0], xdat[-1]])
plt.ylim([0, 100])
plt.gca().set_aspect('equal')

plt.axvline(x=0, color='b')
plt.text(0.3, 80, 'Cue', color='b', fontsize=11)

plt.tick_params(labelsize=15)
plt.show()

# Uncomment to save results
# savetime = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
# savename = f"decodingResults_{protocol_name}_S{decodingResults['session']}_{savetime}_{processing}"
# savemat(savename + '.mat', {'decodingResults': decodingResults})

h5py loading failed: Unable to allocate 701. MiB for an array with shape (10800, 128, 133) and data type float32
Both loading methods failed. h5py error: Unable to allocate 701. MiB for an array with shape (10800, 128, 133) and data type float32, scipy error: Please use HDF reader for matlab v7.3 files, e.g. h5py


NotImplementedError: Please use HDF reader for matlab v7.3 files, e.g. h5py